# Lab 1 - Data Manipulation with Pandas


<div>
<img src="../../images/lab01/pandas_logo.png" width="700"/>
</div>

_(Adapted from [CS109a: Introduction to Data Science](https://harvard-iacs.github.io/2019-CS109A/), [Pandas: Getting Started](https://pandas.pydata.org/docs/getting_started/index.html) & [GitHub: pandas_exercises](https://github.com/guipsamora))_


# 1. Quick Overview


In [1]:
import pandas as pd

from pathlib import Path
from typing import List

# Initialize a base path for us to use
BASE_PATH = Path().cwd()

BASE_PATH

PosixPath('/Users/user/GitHub/Information-Retrieval-Exercises/exercises/lab01-intro-to-pandas')

## How is a DataFrame structured?

<div>
<img src="../../images/lab01/pandas_structure.png" width="700"/>
</div>


Getting started with using pandas


In [2]:
df = pd.DataFrame(
    {
        "Name": [
            "Braund, Mr. Owen Harris",
            "Allen, Mr. William Henry",
            "Bonnell, Miss. Elizabeth",
        ],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"],
    }
)

df

,Name,Age,Sex
0,"Braund, Mr. Owen Harris",22,male
1,"Allen, Mr. William Henry",35,male
2,"Bonnell, Miss. Elizabeth",58,female


In [4]:
df[["Age"]]

,Age
0,22
1,35
2,58


When selecting a single column of a pandas **`DataFrame`**, the result is a pandas **`Series`**.


In [5]:
type(df["Age"])

pandas.core.series.Series

A pandas **`Series`** has no column labels, as it is just a single column of a **`DataFrame`**. A Series does have row labels.


In [6]:
# Access the series by the index (row label)
series = df["Age"]

series.loc[series.index % 2 == 0]

0    22
2    58
Name: Age, dtype: int64

## How do we get data inside a DataFrame?

<div>
<img src="../../images/lab01/pandas_read_data.png" width="700"/>
</div>

Pretty simple, just use the (hopefully existing) **`read_<file_extension>`** method:


In [7]:
DATA_PATH = BASE_PATH / "data"

titanic = pd.read_csv(DATA_PATH / "titanic" / "titanic.csv", index_col=0)
titanic

,survived,pclass,name,sex,age,sibSp,parch,ticket,fare,cabin,embarked
passengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


The great thing about this modular approach, is that if we know that the file extension maps one to one to an existing pandas method, then we have nothing to worry about.

_Note: (If we were working with something like `xls` or `xlsx`, which are 'Microsoft Excel Open XML,' we would need map to the according method)_


In [8]:
def load_data(data_path: Path) -> List[pd.DataFrame]:
    """Loads all readable data files from a given directory into pandas DataFrames.

    Args:
        data_path (Path): Path object representing the base directory
            containing the data files.

    Returns:
        List[pd.DataFrame]: A list of pandas DataFrames, one per successfully
            loaded file.
    """
    files_found = [path for path in data_path.glob("*") if path.is_file()]

    result = []
    for found in files_found:
        # Give us the file extension (.<ext>) and then remove the '.' leaving us only with <ext>
        file_extension = found.suffix.lstrip(".")

        read_method = getattr(pd, f"read_{file_extension}")
        if callable(read_method):
            result.append(read_method(found))

    return result

In [9]:
data = load_data(DATA_PATH / "titanic")

print(f"Found: '{len(data)}' DataFrames")
data[2]

Found: '3' DataFrames


,passengerId,survived,pclass,name,sex,age,sibSp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


To check on how pandas interpreted each of the column data types can be done by requesting the pandas **`dtypes`** attribute:


In [10]:
titanic.dtypes

survived      int64
pclass        int64
name         object
sex          object
age         float64
sibSp         int64
parch         int64
ticket       object
fare        float64
cabin        object
embarked     object
dtype: object

Here, for each of the columns, the used data type is enlisted. The data types in this **`DataFrame`** are integers (**`int64`**), floats (**`float64`**) and strings (**`object`**).

What is the (potential) consequence of **`dtype`** being **`object`** for strings? <br>
$\rightarrow$ Might not be the fastest approach & we also can't simply apply numerical operations

###### _Note:_ (_Starting pandas v3.0 the pyarrow string type will actually be the default string data type, for more see:_ https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#background)


In [11]:
import pandas as pd

n = 1_000_000
series_obj = pd.Series(["hello"] * n, dtype=object) # Numpy ndarray
series_arrow = series_obj.astype("string[pyarrow]") # pyarrow string

print(series_obj.dtype)
print(series_arrow.dtype)

print("\nBenchmarking .str.upper() ...")

print("object dtype:")
%timeit series_obj.str.upper()

print("string[pyarrow] dtype:")
%timeit series_arrow.str.upper()

object
string

Benchmarking .str.upper() ...
object dtype:
76.6 ms ± 333 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
string[pyarrow] dtype:
9.94 ms ± 313 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## How can you work with pandas DataFrames?


The Titanic data set consists of the following data columns:

- **`PassengerId`**: Id of every passenger (implicit index of the row).

- **`Survived`**: Indication whether passenger survived. 0 for yes and 1 for no.

- **`Pclass`**: One out of the 3 ticket classes: Class 1, Class 2 and Class 3.

- **`Name`**: Name of passenger.

- **`Sex`**: Gender of passenger.

- **`Age`**: Age of passenger in years.

- **`SibSp`**: Number of siblings or spouses aboard.

- **`Parch`**: Number of parents or children aboard.

- **`Ticket`**: Ticket number of passenger.

- **`Fare`**: Indicating the fare.

- **`Cabin`**: Cabin number of passenger.

- **`Embarked`**: Port of embarkation.


<div>
<img src="../../images/lab01/pandas_columns.png" width="700"/>
</div>


In [12]:
age_status = titanic[["Age", "Survived"]]

KeyError: "None of [Index(['Age', 'Survived'], dtype='object')] are in the [columns]"

Why did this fail? Are we sure we got the columns right?


In [13]:
titanic.columns

Index(['survived', 'pclass', 'name', 'sex', 'age', 'sibSp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked'],
      dtype='object')

The columns don't match our expected specifications, but we can adjust this easily


In [18]:
# Change the columns to match our specification from above
titanic.columns = titanic.columns.str.capitalize()

age_status = titanic[["Age", "Survived"]]

age_status.head(15)

,Age,Survived
passengerId,,
1,22.0,0
2,38.0,1
3,26.0,1
4,35.0,1
5,35.0,0
6,NaN,0
7,54.0,0
8,2.0,0
9,27.0,1


<div>
<img src="../../images/lab01/pandas_rows.png" width="700"/>
</div>


In [19]:
# Let's see how many passengers were on the titanic, who were older than 35 years at that time
above_35 = titanic[titanic["Age"] > 35]
above_35

,Survived,Pclass,Name,Sex,Age,Sibsp,Parch,Ticket,Fare,Cabin,Embarked
passengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
12,1,1,"Bonnell, Miss Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.0000,NaN,S
872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
874,0,3,"Vander Cruyssen, Mr. Victor",male,47.0,0,0,345765,9.0000,NaN,S


The condition inside the selection brackets **`titanic["Age"] > 35`** checks for which rows the **`Age`** column has a value larger than 35, so:

```py
titanic["Age"] > 35
0      False
1       True
2      False
3      False
4      False
       ...
886    False
887    False
888    False
889    False
890    False
Name: Age, Length: 891, dtype: bool
```

returns a pandas **`Series`** of boolean values, which are either **`True`** or **`False`**, with the same number of rows as the original **`DataFrame`**.


<div>
<img src="../../images/lab01/pandas_specify.png" width="700"/>
</div>


Let's say, we are only interested in the names of passengers that were older than 35 years


In [20]:
above_35_names = titanic.loc[titanic["Age"] > 35, "Name"]
above_35_names

passengerId
2      Cumings, Mrs. John Bradley (Florence Briggs Th...
7                                McCarthy, Mr. Timothy J
12                               Bonnell, Miss Elizabeth
14                           Andersson, Mr. Anders Johan
16                      Hewlett, Mrs. (Mary D Kingcome) 
                             ...                        
866                             Bystrom, Mrs. (Karolina)
872     Beckwith, Mrs. Richard Leonard (Sallie Monypeny)
874                          Vander Cruyssen, Mr. Victor
880        Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)
886                 Rice, Mrs. William (Margaret Norton)
Name: Name, Length: 217, dtype: object

In this case, a subset of both rows and columns is made in one go and just using selection brackets **`[]`** is not sufficient anymore. The **`loc`**/**`iloc`** operators are required in front of the selection brackets **`[]`**.

When using **`loc`**/**`iloc`**, the part before the comma is the **rows** you want, and the part after the comma is the **columns** you want to select.

For both the part before and after the comma, you can use a single label, a **list** of labels, a **slice** of labels, a **conditional expression** or a **colon**. Using a colon specifies you want to select all rows or columns.


In [21]:
titanic.iloc[9:25, 2:5]

,Name,Sex,Age
passengerId,,,
10,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0
11,"Sandstrom, Miss Marguerite Rut",female,4.0
12,"Bonnell, Miss Elizabeth",female,58.0
13,"Saundercock, Mr. William Henry",male,20.0
14,"Andersson, Mr. Anders Johan",male,39.0
15,"Vestrom, Miss Hulda Amanda Adolfina",female,14.0
16,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0
17,"Rice, Master Eugene",male,2.0
18,"Williams, Mr. Charles Eugene",male,NaN


When specifically interested in certain rows and/or columns **based on their position** in the table, use the **`iloc`** operator in front of the selection brackets **`[]`**


In [22]:
# Of course you can also mix the ideas of iloc and loc, which makes it easier to avoid accidental column selections
titanic.iloc[0:3, titanic.columns.get_loc("Name")] = "anonymous"

titanic

,Survived,Pclass,Name,Sex,Age,Sibsp,Parch,Ticket,Fare,Cabin,Embarked
passengerId,,,,,,,,,,,
1,0,3,anonymous,male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,anonymous,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,anonymous,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


<div>
<img src="../../images/lab01/pandas_groupby.png" width="700"/>
</div>

What is the average age for male versus female Titanic passengers?


In [24]:
titanic[["Sex", "Age"]].groupby("Sex").max()

,Age
Sex,
female,63.0
male,80.0


Since we are interested in the average age for each gender, we first do a subselection on these two columns(**`titanic[["Sex", "Age"]]`**). Next, we apply the **`groupby()`** method on the **`Sex`** column to create one group per category (and since there are only two values in the column, we will have two groups created). Last, the average each for each category is calculated and returned.

This approach is the general **`split-apply-combine`** pattern:

- **Split** the data into groups
- **Apply** a function to each group independently
- **Combine** the results into a data structure


In [27]:
# Why can't we just apply the groupby operation directly?
titanic.groupby("Sex").mean()

TypeError: agg function failed [how->mean,dtype->object]

In [29]:
# Recalling the dtypes, we are applying a numeric operation on types that are incompatible with the operation.
titanic.info()

# We can avoid this by passing `numeric_only=True`
titanic.groupby("Sex").mean(numeric_only=True)

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   Sibsp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


,Survived,Pclass,Age,Sibsp,Parch,Fare
Sex,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


<div>
<img src="../../images/lab01/pandas_count.png" width="700"/>
</div>
What is the number of passengers in each of the cabin classes?


In [31]:
titanic["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

The **`value_counts()`** method counts the number of records for each distinct value in a column. It is a shortcut method, as it is actually a groupby operation in combination with counting of the number of records within each group:

```py
titanic.groupby("Pclass")["Pclass"].count()
Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64
```


# 2. Exercises

Summary of operations & Documentation available at: https://pandas.pydata.org/docs/user_guide/10min.html


### Give the percentage of survivors


### What is the average age and gender of the survivors compared to the people that didn't survive?

**Note**: Pandas does automatically exclude NaN numbers from aggregation functions. If the only value in the column is NaN, then we must take the aggregate value of an empty set, which is results in NaN


### Create a new column, called `AgeGroup`, which classifies the person based on their **`Age`** as follows:

- If 0 < **`Age`** <= 1, then classify them as **`Infant`**
- If 1 < **`Age`** <= 3, then classify them as **`Toddler`**
- If 3 < **`Age`** <= 12, then classify them as **`Child`**
- If 12 < **`Age`** <= 18, then classify them as **`Teen`**
- if 18 < **`Age`** <= 30, then classify them as **`YoungAdult`**
- If 30 < **`Age`** <= 50, then classify them as **`Adult`**
- If 50 < **`Age`** <= 80, then classify them as **`Senior`**
- If 80 < **`Age`** <= 130, then classify them as **`Urgestein`**


### Now, measure the survival rate by age group


## Working with Chipotle


Load the **`chipotle.csv`** from the **`data/chipotle`** directory.

Tip: perhaps some detail in the documentation is necessary to load the file.


In [ ]:
chipo_path = ...

### Inspect the first 10 entries


### What is the number of columns in the dataset?


### Print the name of all the columns.


### How is the dataset indexed?


### What were the ten most-ordered items? And how often were they ordered?


In [ ]:
# Solution: Chicken Bowl: 761, ...

### How many items were ordered in total?


### How much was the revenue for the period in the dataset?

Tip: if you are running into issues, check the type of the column(s) that you need to work with. Perhaps you need preprocessing before proceeding with some steps


In [ ]:
revenue = ...

print("Revenue was: $" + str(round(revenue, 2)))
# Solution: Revenue was: $39237.02

### How many orders were made in the period?


### What is the lowest, average, and highest revenue per order?


In [ ]:
# Solution: mean: 21.394...; min: 10.08; max: 1074.24; median: 16.65

### How many different items are sold?


In [ ]:
# Solution: 50

### How many products cost more than $10.00 ?

Tip: Inspect the item_price column for a specific item to see how the price and item_name relate to each other.


In [ ]:
# item_name and choice_description appear is pairs multiple times, so we must drop them to avoid falsifying our results

# Solution: 707 rows

### How many different product prices exist?


In [ ]:
# Solution: 37

### What is the quantity of the most expensive **item** ordered?


### How many times did someone order more than one Canned Soda?


### List the full order of the person that wanted the most canned sodas.


### (Advanced): Create a profitability report about the menu, which includes for each item:

- Total quantity sold
- Total revenue generated
- Number of **unique** orders containing the item
- Average selling price per unit

At the end, rank the items by their revenue contribution (% of total revenue)


### (Advanced): Are there price inconsistencies? If so, list them.

Background: Some items on our menu may have been sold at different prices, e.g. depending on add-ons. Find all items that have more than one unique price and list their price ranges.
